In [ ]:
%reset

# 1. Imports

In [1]:
import mne
import time
import os
import custom_modules.xdf_to_fif_converter as xtfc
import custom_modules.preprocessing_ptu as prep
import custom_modules.file_handling as  fh

import importlib
importlib.reload(xtfc)
importlib.reload(prep)
importlib.reload(fh)

<module 'custom_modules.file_handling' from 'C:\\Users\\peter\\Documents\\Code\\master-thesis\\03_analysis\\custom_modules\\file_handling.py'>

# 2. Constants

In [2]:
data_path = 'D:/Diplomarbeit_data/eeg/'
subjects = ['A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07' , 'A08', 'A09', 'A10']
trial_type_markers = ['LTR-s', 'LTR-l','RTL-s', 'RTL-l', 'TTB-s', 'TTB-l', 'BTT-s', 'BTT-l']

mne.set_log_level('WARNING') #'INFO' 'WARNING'

# 3. XDF --> FIF
The following cell reads all xdf-files containing the raw eeg and loads them into the mne structure and stores them as .fif-files.

In [ ]:
help(xtfc.xdf_to_fif)

In [ ]:
# Get source and destination path + create destintation folder if it does not exist.
src_path, dst_path = fh.gen_paths(pth=data_path, src_fldr='raw', dst_fldr='raw_fif')

# Iterate over each subject and extract the streams
start = time.time()
for i, subject in enumerate(subjects):
    print(f'Extracting subject {subject}', end=' ')

    # Call the xdf_to_fif function from the xtfc module which takes care of converting the xdf files to fif files:
    xtfc.xdf_to_fif(src_path, dst_path, subject)

    print()

print(f'Finished reading, took me {round(time.time()-start)} seconds...')

# 4. Concatenate all fif files for each subject

In [ ]:
help(prep.concat_fifs)

In [ ]:
# Get source and destination path + create destintation folder if it does not exist.
src_path, dst_path = fh.gen_paths(pth=data_path, src_fldr='raw_fif', dst_fldr='concat')

# Iterate over each subject and extract the streams
start = time.time()
for i, subject in enumerate(subjects):
    print(f'Reading all fif files for subject {subject}', end=' ')

    # Concatenate all raw files for the paradigm and the eye paradigm:
    prep.concat_fifs(src=src_path, dst=dst_path, sbj=subject, paradigm='paradigm')
    prep.concat_fifs(src=src_path, dst=dst_path, sbj=subject, paradigm='eye')
    print()

print(f'Finished concatenating, took me {round(time.time() - start)} seconds...')

# 5. Filter  the data (HP 0.4 Hz and Notch 50 Hz)

In [ ]:
help(prep.filter_fifs)

In [ ]:
# Get source and destination path + create destintation folder if it does not exist.
src_path, dst_path = fh.gen_paths(pth=data_path, src_fldr='concat', dst_fldr='filtered')

# Iterate over each subject and extract the streams
start = time.time()
for i, subject in enumerate(subjects):
    print(f'Filtering raw EEG for subject {subject}')

    # Concatenate all raw files for the paradigm and the eye paradigm:
    prep.filter_fifs(src=src_path, dst=dst_path, sbj=subject, paradigm='paradigm')
    prep.filter_fifs(src=src_path, dst=dst_path, sbj=subject, paradigm='eye')

print(f'Finished filtering, took me {round(time.time() - start)} seconds...')

# 6. Interpolate bad channels

In this cell the bad channels are interpolated based on visual inspection. The visual inspection was performed prior and the channels are stored in ***/dataframes/preprocessing/bad_channels.csv***

In [ ]:
help(prep.interpolate_bads)

In [ ]:
# Get source and destination path + create destintation folder if it does not exist.
src_path, dst_path = fh.gen_paths(pth=data_path, src_fldr='filtered', dst_fldr='bad_ch_interpolated')

# Iterate over each subject and extract the streams
start = time.time()
for i, subject in enumerate(subjects):
    print(f'Interpolating bad channels for subject {subject}')

    # Interpolate bad channels for all raw files for the paradigm and the eye paradigm:
    prep.interpolate_bads(src=src_path, dst=dst_path, sbj=subject, paradigm='paradigm')
    prep.interpolate_bads(src=src_path, dst=dst_path, sbj=subject, paradigm='eye')

print(f'Finished interpolating, took me {round(time.time() - start)} seconds...')

# 7. CAR

In [ ]:
# Get source and destination path + create destintation folder if it does not exist.
src_path, dst_path = fh.gen_paths(pth=data_path, src_fldr='bad_ch_interpolated', dst_fldr='car_filtered')

# Iterate over each subject and extract the streams
start = time.time()
for i, subject in enumerate(subjects):
    print(f'Re-referencing for subject {subject}')

    # Concatenate all raw files for the paradigm:
    prep.car(src=src_path, dst=dst_path, sbj=subject, paradigm='paradigm')

print(f'Finished interpolating, took me {round(time.time() - start)} seconds...')

# 8. Mark bad dataspans
Mark bad dataspans due to user errors in paradigm.

In [ ]:
# Get source and destination path + create destintation folder if it does not exist.
src_path, dst_path = fh.gen_paths(pth=data_path, src_fldr='car_filtered', dst_fldr='bad_dataspans_marked')

# Iterate over each subject and extract the streams
start = time.time()
for i, subject in enumerate(subjects):
    print(f'Marking bad dataspans for outlier detection for subject {subject}')

    # Concatenate all raw files for the paradigm:
    prep.mark_bad_dataspans(src=src_path, dst=dst_path, sbj=subject, paradigm='paradigm')

print(f'Finished interpolating, took me {round(time.time() - start)} seconds...')

# 9. Epoching for outlier detection

In [ ]:
# Get source and destination path + create destintation folder if it does not exist.
src_path, dst_path = fh.gen_paths(pth=data_path, src_fldr='bad_dataspans_marked', dst_fldr='epoched_for_outlier_detection')

# Iterate over each subject and extract the streams
start = time.time()
for i, subject in enumerate(subjects):
    print(f'Epoching for outlier detection for subject {subject}')

    # Concatenate all raw files for the paradigm:
    prep.epoch_for_outlier_detection(src=src_path, dst=dst_path, sbj=subject, paradigm='paradigm')

print(f'Finished interpolating, took me {round(time.time() - start)} seconds...')

## 9.1. Visualize epochs

In [8]:
# Get source and destination path + create destintation folder if it does not exist.
src_path, dst_path = fh.gen_paths(pth=data_path, src_fldr='epoched_movement_aligned', dst_fldr='epoched_for_outlier_detection')

sbj_to_vis = 'A01'

# Visualize epochs for subject:
epochs = prep.vis_epochs_for_sbj(src=src_path, sbj=sbj_to_vis)
fig = epochs[9].plot(picks=['eeg', 'eog'])

# 10. Lowpass filtering

In [ ]:
# Get source and destination path + create destintation folder if it does not exist.
src_path, dst_path = fh.gen_paths(pth=data_path, src_fldr='bad_dataspans_marked', dst_fldr='lowpass_filtered')

# Iterate over each subject and extract the streams
start = time.time()
for i, subject in enumerate(subjects):
    print(f'Lowpass filter for subject {subject}')

    # Lowpass filter all raw files for the paradigm:
    prep.lowpass_filter(src=src_path, dst=dst_path, sbj=subject, paradigm='paradigm')

print(f'Finished lowpass filtering, took me {round(time.time() - start)} seconds...')

# 11. Epoch and resample:

## 11.1. Cue-aligned

In [ ]:
# Get source and destination path + create destintation folder if it does not exist.
src_path, dst_path = fh.gen_paths(pth=data_path, src_fldr='lowpass_filtered', dst_fldr='epoched_cue_aligned')

# Iterate over each subject and extract the streams
start = time.time()
for i, subject in enumerate(subjects):
    print(f'Epoching cue-aligned for subject {subject}')

    # Epoch all raw files for the paradigm:
    prep.epoch_and_resample(src=src_path, dst=dst_path, sbj=subject, paradigm='paradigm', cue_aligned=True)

print(f'Finished epoching, took me {round(time.time() - start)} seconds...')

## 11.2. Movement-onset aligned

In [ ]:
# Get source and destination path + create destintation folder if it does not exist.
src_path, dst_path = fh.gen_paths(pth=data_path, src_fldr='lowpass_filtered', dst_fldr='epoched_movement_aligned')

# Iterate over each subject and extract the streams
start = time.time()
for i, subject in enumerate(subjects):
    print(f'Epoching movement-aligned for subject {subject}')

    # Epoch all raw files for the paradigm:
    prep.epoch_and_resample(src=src_path, dst=dst_path, sbj=subject, paradigm='paradigm', cue_aligned=False)

print(f'Finished epoching, took me {round(time.time() - start)} seconds...')